In [3]:
from __future__ import print_function, division
import pprint
import os 
import fnmatch
#import errno
import os.path as osp
import json
#import re
import glob as gb
from six import string_types
from nipype.interfaces.base import Bunch

import utils
reload(utils)

<module 'utils' from 'utils.pyc'>

In [4]:
pp = pprint.PrettyPrinter(indent=2)

In [5]:
#- testing

test_case_105a = {
    'base_dir': '/home/jb/data/bids/ds105',
    'model_pattern': "ds-105_sub*_model.json"
}
test_case_105b = {
    'base_dir': '/home/jb/data/bids/ds105',
    'model_pattern': "ds-105_level-all_model.json"
}
test_case_005 = {
    'base_dir': '/home/jb/data/bids/ds005',
    'model_pattern': "*_model.json"
}

test_case = test_case_005
#test_case = test_case_105a
#test_case = test_case_105b

base_dir, model_pattern = test_case['base_dir'], test_case['model_pattern'] 

all_jsons = utils.glob_recursive(base_dir, model_pattern)
print(all_jsons)
json_model=all_jsons[0]

['/home/jb/data/bids/ds005/models/ds-005_model.json']


In [6]:
data = utils.get_json_model_Ydata(all_jsons[0], level='Run', verbose=utils.VERB['info'])

base dir /home/jb/data/bids/ds005
json_model /home/jb/data/bids/ds005/models/ds-005_model.json


In [7]:
pp.pprint(data[:min(10, len(data))])

[ '/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-01_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-02_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-03_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-03/func/sub-03_task-mixedgamblestask_run-01_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-03/func/sub-03_task-mixedgamblestask_run-03_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-03/func/sub-03_task-mixedgamblestask_run-02_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-08/func/sub-08_task-mixedgamblestask_run-02_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-08/func/sub-08_task-mixedgamblestask_run-01_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-08/func/sub-08_task-mixedgamblestask_run-03_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-12/func/sub-12_task-mixedgamblestask_run-02_bold.nii.gz']


In [8]:
model_pattern = "*_model.json"
model_data = utils.get_funcs_models(base_dir, model_pattern, verbose=utils.VERB['none'])

In [9]:
datafile = data[0]
model_dict = model_data[datafile]
pp.pprint(model_dict)

{ u'Columns': { u'dummy': { u'HRFmodelling': u'Gamma',
                            u'Level': -1,
                            u'Variable': u'respcat'},
                u'gain': { u'HRFmodelling': u'Gamma',
                           u'Level': 0,
                           u'ModulationOrder': 1,
                           u'ModulationVar': u'RT',
                           u'Variable': u'respcat'},
                u'loss': { u'HRFmodelling': u'Gamma',
                           u'Level': 1,
                           u'ModulationVar': u'RT',
                           u'Variable': u'respcat'}},
  u'Contrasts': { u'pos_vs_neg_contrast': { u'Columns': [u'gain', u'loss'],
                                            u'Weights': [1, -1]}},
  u'DependentVariable': u'task-mixedgamblestask',
  u'Level': u'Run'}


In [10]:
import pandas as pd
tsv_file = datafile.split('_bold.nii')[0] + '_events.tsv'
assert osp.isfile(tsv_file), "{} is not a file ".format(tsv_file)
print(tsv_file)

/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-01_events.tsv


In [11]:
dic = utils._get_json_dict_from_tsv_file(tsv_file)
#utils._get_tsv_lines(dic, 'trial_type', 'chair')

In [12]:
model_dict = model_data[datafile]
print(model_dict.keys())
regressors = model_dict['Columns']
pp.pprint(regressors)

[u'Contrasts', u'DependentVariable', u'Columns', u'Level']
{ u'dummy': { u'HRFmodelling': u'Gamma', u'Level': -1, u'Variable': u'respcat'},
  u'gain': { u'HRFmodelling': u'Gamma',
             u'Level': 0,
             u'ModulationOrder': 1,
             u'ModulationVar': u'RT',
             u'Variable': u'respcat'},
  u'loss': { u'HRFmodelling': u'Gamma',
             u'Level': 1,
             u'ModulationVar': u'RT',
             u'Variable': u'respcat'}}


In [13]:
list_dicts = utils.get_run_conditions(datafile, model_dict)
condition_names = list_dicts.keys()

In [20]:
#pp.pprint(list_dicts)
for datafile in data[:min(10, len(data))]:
    #print(datafile)
    try: 
        nipype_info = utils.get_nipype_run_info(datafile, model_dict, verbose=utils.VERB['none'])
    except:
        print(datafile, ' : has a pb ... a condition has no event ')

/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-02_bold.nii.gz  : has a pb ... a condition has no event 
/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-03_bold.nii.gz  : has a pb ... a condition has no event 
/home/jb/data/bids/ds005/sub-03/func/sub-03_task-mixedgamblestask_run-01_bold.nii.gz  : has a pb ... a condition has no event 
/home/jb/data/bids/ds005/sub-03/func/sub-03_task-mixedgamblestask_run-03_bold.nii.gz  : has a pb ... a condition has no event 
/home/jb/data/bids/ds005/sub-08/func/sub-08_task-mixedgamblestask_run-01_bold.nii.gz  : has a pb ... a condition has no event 
/home/jb/data/bids/ds005/sub-08/func/sub-08_task-mixedgamblestask_run-03_bold.nii.gz  : has a pb ... a condition has no event 


In [15]:
#print(datafile)
#pp.pprint(nipype_info)
#print(nipype_info)

In [16]:
dict_regressors = utils.get_run_conditions(datafile, model_dict)#, verbose=verbose)
condition_names = dict_regressors.keys() 
#print(dict_regressors)
bunch = utils.make_nipype_bunches(condition_names, dict_regressors)

In [17]:
bunch

Bunch(conditions=[u'dummy', u'gain', u'loss'], durations=[[3, 3, 3, 3, 3, 3, 3, 3, 3], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]], onsets=[[0.0, 4.0, 8.0, 12.0, 16.0, 84.0, 184.0, 330.0, 448.0], [48.0, 56.0, 60.0, 64.0, 72.0, 76.0, 88.0, 98.0, 132.0, 136.0, 166.0, 194.0, 216.0, 240.0, 250.0, 262.0, 294.0, 322.0, 326.0, 364.0, 400.0, 404.0, 426.0, 440.0, 444.0, 460.0], [24.0, 28.0, 32.0, 36.0, 40.0, 44.0, 94.0, 102.0, 106.0, 110.0, 120.0, 146.0, 150.0, 154.0, 162.0, 176.0, 180.0, 200.0, 204.0, 208.0, 212.0, 220.0, 226.0, 236.0, 244.0, 258.0, 274.0, 282.0, 286.0, 298.0, 304.0, 316.0, 334.0, 342.0, 348.0, 358.0, 368.0, 372.0, 378.0, 384.0, 390.0, 396.0, 408.0, 412.0, 422.0, 432.0, 436.0, 452.0, 468.0, 474.0]], pmod=[[], [Bunch(name=u'RT', param=[1.4259999999999999, 2.2160000000000002, 1.9530000000000001

### copy a ds tree with only the json and tsv files

In [18]:
source_dir = '/home/jb/data/bids/ds005'
dest_dir = '/home/jb/code/bids2pype/bids2pype/tests/data/ds005'
# utils.create_empty_bids(source_dir, dest_dir, ['*.json','*.tsv'])